In [49]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [50]:
#  !conda install -c conda-forge vit-pytorch -y

In [51]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from vit_pytorch import ViT

In [52]:
# Define custom dataset
class Sat4Dataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32)
        label = self.labels[idx].astype(np.int64)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [53]:
# Load and preprocess data
def image_processing(df_x):
    reshaped_X = df_x.values.reshape(-1, 28, 28, 4).astype(float)
    reshaped_X_new = reshaped_X / 255.0
    reshaped_X_rgb = reshaped_X_new[:, :, :, :3]
    return reshaped_X_rgb

In [54]:
def label_processing(df_y):
    df_y['Labels'] = "NA"
    for ix in range(len(df_y)):
        if df_y.iloc[ix, 0] == 1:
            df_y.iloc[ix, 4] = "Barren Land"
        elif df_y.iloc[ix, 1] == 1:
            df_y.iloc[ix, 4] = "Trees"
        elif df_y.iloc[ix, 2] == 1:
            df_y.iloc[ix, 4] = "Grassland"
        else:
            df_y.iloc[ix, 4] = "None"
    df_y = df_y['Labels']
    label_map = {"Barren Land": 0, "Trees": 1, "Grassland": 2, "None": 3}
    labels = df_y.map(label_map).values
    return labels

In [55]:
path = "/kaggle/input/deepsat4-subsets/"

In [56]:
df_x_train = pd.read_csv(path + "chunk_x_train_1.csv")
df_y_train = pd.read_csv(path + "chunk_y_train_1.csv")
df_x_test = pd.read_csv(path + "chunk_x_test_1.csv")
df_y_test = pd.read_csv(path + "chunk_y_test_1.csv")

In [57]:
reshaped_x_train_rgb = image_processing(df_x_train)
train_labels = label_processing(df_y_train)

In [58]:
reshaped_x_test_rgb = image_processing(df_x_test)
test_labels = label_processing(df_y_test)

In [59]:
print("Unique train labels:", np.unique(train_labels))
print("Unique test labels:", np.unique(test_labels))

Unique train labels: [0 1 2 3]
Unique test labels: [0 1 2 3]


In [60]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [61]:
train_dataset = Sat4Dataset(reshaped_x_train_rgb, train_labels, transform)
test_dataset = Sat4Dataset(reshaped_x_test_rgb, test_labels, transform)
train_size = len(train_dataset)
test_size = len(test_dataset)

In [62]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [63]:
# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [64]:
# Initialize ViT model, loss function, and optimizer
model = ViT(
    image_size = 224,
    patch_size = 32,
    num_classes = 4,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [65]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move images and labels to GPU
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1/10, Loss: 0.7463813611613516
Epoch 2/10, Loss: 0.695645830858394
Epoch 3/10, Loss: 0.7161111665122649
Epoch 4/10, Loss: 0.6833347265449021
Epoch 5/10, Loss: 0.7413558715673358
Epoch 6/10, Loss: 0.7665075421180871
Epoch 7/10, Loss: 0.7711625590806117
Epoch 8/10, Loss: 0.7887809921408553
Epoch 9/10, Loss: 0.7648241600721998
Epoch 10/10, Loss: 0.770051057762502


In [66]:
# Evaluation
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move images and labels to GPU
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Confusion matrix and classification report
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))

[[2292    1  242  120]
 [   0  795   20 1199]
 [ 672   17  836  325]
 [ 104  503  187 2687]]
              precision    recall  f1-score   support

           0       0.75      0.86      0.80      2655
           1       0.60      0.39      0.48      2014
           2       0.65      0.45      0.53      1850
           3       0.62      0.77      0.69      3481

    accuracy                           0.66     10000
   macro avg       0.66      0.62      0.62     10000
weighted avg       0.66      0.66      0.65     10000

